# Exploratory Analysis - stock

In [28]:
# Stats of interest:
# Greatest volatility one day?
# Num records
# Avg close
# Avg volatility one day
# Growth (total, 1yr, 5yr, etc)

In [29]:
import pandas as pd

In [30]:
rs = pd.read_csv("stock_market_data-AAPL.csv", index_col=0, parse_dates=["Date"])

In [31]:
rs = rs.reindex(index=rs.index[::-1])

In [32]:
rs.head()

,Date,Low,High,Close,Open
0,1999-11-01,77.37,80.69,77.62,80.00
1,1999-11-02,77.31,81.69,80.25,78.00
2,1999-11-03,81.00,83.25,81.50,81.62
3,1999-11-04,80.62,85.37,83.62,82.06
4,1999-11-05,84.00,88.37,88.31,84.62


In [33]:
date, close, ope = rs["Date"], rs["Close"], rs["Open"]

### Light stats

In [34]:
avg_close = rs["Close"].mean()
avg_close

174.74703307664052

In [35]:
len(rs)

5454

In [36]:
# First and last dates of dataset
min(date), max(date)

(Timestamp('1999-11-01 00:00:00'), Timestamp('2021-07-06 00:00:00'))

In [37]:
close.describe()

count    5454.000000
mean      174.747033
std       157.542041
min        13.120000
25%        63.885000
50%       124.390000
75%       215.887500
max       702.100000
Name: Close, dtype: float64

In [38]:
# Growth from earliest to most recent
close[len(close) - 1] - close[0]

64.4

### Deltas

In [42]:
# Delta day to day

In [43]:
delta_l = []
for i in range(1, len(close)):
    delta_l.append((close[i] - close[i - 1]) / close[i])
delta = pd.Series(delta_l)
delta.head()

0    0.032773
1    0.015337
2    0.025353
3    0.053108
4    0.083636
dtype: float64

In [44]:
delta.describe()

count    5453.000000
mean       -0.001353
std         0.094810
min        -5.889755
25%        -0.010479
50%         0.000887
75%         0.013064
max         0.122075
dtype: float64

In [45]:
delta.median()

0.0008872188299150144

In [46]:
# This is telling me the mean delta is negative, but the median is positive? That's interesting. And I think ok -
# 50% of the data points above, 50% below. And the 75% quartile is .013 and the 25% quartile is -.010. 
# Though the min is more extreme than the max.
# And also, we don't have every day continuously from the earliest date to the latest date. 

In [39]:
# Average volatility in a day

In [40]:
day_delta = close - ope
day_delta.head()

0   -2.38
1    2.25
2   -0.12
3    1.56
4    3.69
dtype: float64

In [41]:
day_delta.mean(), day_delta.median()

(-0.014265511551154738, 0.039999999999995595)

In [47]:
# Next Steps: visualization? 
# i.e. visualize the RSMEs

# strip the year of the dates, and then count the frequencies???

In [48]:
# Maybe less valuable?

In [49]:
year = date.dt.year
year.head()

0    1999
1    1999
2    1999
3    1999
4    1999
Name: Date, dtype: int64

In [50]:
year.value_counts().sort_index()

1999     43
2000    252
2001    248
2002    252
2003    252
2004    252
2005    252
2006    251
2007    251
2008    253
2009    252
2010    252
2011    252
2012    250
2013    252
2014    252
2015    252
2016    252
2017    251
2018    251
2019    252
2020    253
2021    127
Name: Date, dtype: int64